## 데이터 불러오기

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from matplotlib import font_manager, rc
import matplotlib as mpl
font_path = "C:\\Users\\이혜림\\Desktop\\Bita5/malgun.ttf" 
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc("font",family=font_name)
mpl.rcParams["axes.unicode_minus"]=False

#경고 무시
import warnings

In [71]:
import sys
import os
data_path = os.path.join("..","6th_data")
pre, reg, reg_day, sub = os.listdir(data_path)

In [72]:
regular = pd.read_csv("../Regular_Season_Batter.csv")

## 운 관련 지표 추가

In [73]:
regular['1B']=regular['H']-regular['2B']-regular['3B']-regular['HR']

regular['1b_luck']=regular['1B']/(regular['AB']-regular['HR']-regular['SO'])
regular['2b_luck']=regular['2B']/(regular['AB']-regular['HR']-regular['SO'])
regular['3b_luck']=regular['3B']/(regular['AB']-regular['HR']-regular['SO'])

## 변동성이 큰 지표는 total로 값을 넣어줌

In [74]:
total_1BnAB=[]
total_3BnAB=[]
total_avg=[]

for i in range(len(regular)):
    a=regular.loc[(regular["batter_name"]==regular["batter_name"].iloc[i])&(regular["year"]<regular["year"].iloc[i])]
    total_1BnAB.append(a["1B"].sum()/a["AB"].sum())
    total_3BnAB.append(a["3B"].sum()/a["AB"].sum())
    total_avg.append((a["avg"]*a["AB"]).sum()/a["AB"].sum())

<ipython-input-74-200038192d9b>:7: RuntimeWarning: invalid value encountered in longlong_scalars
  total_1BnAB.append(a["1B"].sum()/a["AB"].sum())
<ipython-input-74-200038192d9b>:8: RuntimeWarning: invalid value encountered in longlong_scalars
  total_3BnAB.append(a["3B"].sum()/a["AB"].sum())
<ipython-input-74-200038192d9b>:9: RuntimeWarning: invalid value encountered in double_scalars
  total_avg.append((a["avg"]*a["AB"]).sum()/a["AB"].sum())


In [75]:
regular["total_1BnAB"]=total_1BnAB
regular["total_3BnAB"]=total_3BnAB
regular['total_avg']=total_avg

## 과거 n개년 데이터를 feature로 만드는 함수
- 년도별 변동성이 크지 않은 'avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck'에 대해서 진행

In [16]:
raw = regular[:]

In [76]:
def make_dataset(n,data):
    # 과거 n개년에 대해서 실행
    for year in range(1,n+1):
        for j in ['avg', 'G', 'AB', 'R', 'H','2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'SO', 'GDP','SLG', 'OBP', 'E','1b_luck','2b_luck','3b_luck']:
            lag_1_avg=[]
            for i in range(len(data)): 
                if len(data.loc[(data['batter_name']==data['batter_name'].iloc[i])&(data['year']==data['year'].iloc[i]-year)][j])==0:
                    lag_1_avg.append(np.nan)
                else:
                    lag_1_avg.append(data.loc[(data['batter_name']==data['batter_name'].iloc[i])&(data['year']==data['year'].iloc[i]-year)][j].iloc[0])

            data[f'lag_{year}_'+j]=lag_1_avg
            print(f"과거{year}년 {j} 칼럼 완성")
    return data

In [77]:
data = make_dataset(3,regular)

avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E
1b_luck
2b_luck
3b_luck
avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E
1b_luck
2b_luck
3b_luck
avg
G
AB
R
H
2B
3B
HR
TB
RBI
SB
CS
BB
HBP
SO
GDP
SLG
OBP
E
1b_luck
2b_luck
3b_luck


## 팀, 포지션 범주화

In [78]:
# 팀 범주화
data = pd.concat([data,pd.get_dummies(data["team"])], axis=1)

In [50]:
# 포지션 범주화
data = pd.concat([data,pd.get_dummies(data["position"])], axis=1)

## train set에서 제거해야할 칼럼

In [108]:
# total, 최근 1,2,3년 변수들과 age변수 반영
input_columns = list(data.columns)[34:]+["age","OPS"]

## 2015년 이후 데이터만 반영 VS 전체 데이터셋을 모두 반영
- 2015년 이전에 데이터는 그 이후 데이터와 경향성이 다르기 때문에 2015년 이후 데이터만 반영

In [195]:
# 2015년 이후 데이터로만 구성된 데이터셋
data_2015 = data.loc[data["year"]>=2015,]
data_2015 = data_2015.loc[data["AB"]>10,]

In [203]:
raw = data[:]

In [204]:
# 전체 데이터셋
data = data.loc[data["AB"]>10,]

## train,test set 분리

In [205]:
# 전체년도
train = data.loc[data["year"]!=2018,input_columns]
test = data.loc[data["year"]==2018,input_columns]

# OPS에서 null인 값 제거
train = train.loc[train["OPS"].dropna().index,:]
test = test.loc[test["OPS"].dropna().index,:]

In [206]:
# 2015년 이후
train_2015 = data_2015.loc[data_2015["year"]!=2018,input_columns]
test_2015 = data_2015.loc[data_2015["year"]==2018,input_columns]

# OPS에서 null인 값 제거
train_2015 = train_2015.loc[train_2015["OPS"].dropna().index,:]
test_2015 = test_2015.loc[test_2015["OPS"].dropna().index,:]

In [207]:
train_2015_X = train_2015.drop(["OPS"], axis=1)
train_2015_y = train_2015["OPS"]

train_X = train.drop(["OPS"], axis=1)
train_y = train["OPS"]

In [208]:
test_2015_X = test_2015.drop(["OPS"], axis=1)
test_2015_y = test_2015["OPS"]

test_X = test.drop(["OPS"], axis=1)
test_y = test["OPS"]

## XGBoost를 통한 모델링

### 2015년 이후 데이터로만

In [136]:
import xgboost as xgb

In [218]:
xgb_2015 = xgb.XGBRegressor(silent=False, objective = 'reg:squarederror',gamma = 0.2,
                                   max_depth=14, random_state = 1, n_estimators=1000,learning_rate = 0.1)
xgb_2015.fit(train_2015_X,train_2015_y, eval_set=[(test_2015_X,test_2015_y)],
             early_stopping_rounds=100,verbose=5)

[13:14:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.26726
Will train until validation_0-rmse hasn't improved in 100 rounds.
[5]	validation_0-rmse:0.20137
[10]	validation_0-rmse:0.17229
[15]	validation_0-rmse:0.16146
[20]	validation_0-rmse:0.15755
[25]	validation_0-rmse:0.15653
[30]	validation_0-rmse:0.15622
[35]	validation_0-rmse:0.15614
[40]	validation_0-rmse:0.15613
[45]	validation_0-rmse:0.15613
[50]	validation_0-rmse:0.15614
[55]	validation_0-rmse:0.15615
[60]	validation_0-rmse:0.15615
[65]	validation_0-rmse:0.15615
[70]	validation_0-rmse:0.15615
[75]	validation_0-rmse:0.15615
[80]	validation_0-rmse:0.15615
[85]	validation_0-rmse:0.

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=14,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

### 모든 데이터로

In [215]:
xgb_all = xgb.XGBRegressor(silent=False, objective = 'reg:squarederror',gamma = 0.2,
                                   max_depth=14, random_state = 1, n_estimators=1000,learning_rate = 0.1)

xgb_all.fit(train_X,train_y, eval_set=[(test_X,test_y)],
             early_stopping_rounds=100,verbose=5)

[13:12:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.24825
Will train until validation_0-rmse hasn't improved in 100 rounds.
[5]	validation_0-rmse:0.16549
[10]	validation_0-rmse:0.15167
[15]	validation_0-rmse:0.14919
[20]	validation_0-rmse:0.14893
[25]	validation_0-rmse:0.14886
[30]	validation_0-rmse:0.14883
[35]	validation_0-rmse:0.14883
[40]	validation_0-rmse:0.14883
[45]	validation_0-rmse:0.14883
[50]	validation_0-rmse:0.14883
[55]	validation_0-rmse:0.14883
[60]	validation_0-rmse:0.14883
[65]	validation_0-rmse:0.14883
[70]	validation_0-rmse:0.14883
[75]	validation_0-rmse:0.14883
[80]	validation_0-rmse:0.14883
[85]	validation_0-rmse:0.

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=14,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

-> 2015년 이후 데이터 만을 반영하는 것이 성능이 더 좋음